In [1]:
import sys
import os
from pathlib import Path
cwd = Path(os.getcwd()).parent.as_posix()
if cwd not in sys.path:
    sys.path.append(cwd)

from enum import Enum

import numpy as np
import tensorflow as tf
from typing import Tuple, Union, Callable, Optional, Any
import tensorflow_gnn as tfgnn
from tensorflow_gnn.models.gcn.gcn_conv import GCNHomGraphUpdate

from sds.utils.common import IntOrTuple, int_to_tuple
from sds.utils.utils import calc_3d_graph_elevated


2022-11-20 12:14:51.513379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'sds.utils.common'

In [2]:
class RotHeadType(Enum):
    Conv2d = 'conv2d'
    Conv3d = 'conv3d'

ROT_HEAD_TYPE_VALUES = [ht.value for ht in RotHeadType]
Activation = Callable[[Any], Any]


In [3]:
def block_down(x: Any, strides: IntOrTuple, ch_in: int, ch_out: int = 0, kernel_size: IntOrTuple = 5,
               act: Any = tf.nn.relu) -> Tuple[Any, int]:
    strides = int_to_tuple(strides)
    if not ch_out:
        ch_out = ch_in * strides[0] * strides[1]
    x = tf.keras.layers.Conv2D(ch_out, kernel_size, strides, 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if act is not None:
        x = act(x)
    return x, ch_out


def block_up(x: Any, strides: IntOrTuple, ch_in: int, ch_out: int = 0, kernel_size: IntOrTuple = 4,
             act: Any = tf.nn.relu) -> Tuple[Any, int]:
    strides = int_to_tuple(strides)
    if not ch_out:
        ch_out = ch_in // (strides[0] * strides[1])
    x = tf.keras.layers.Conv2DTranspose(ch_out, kernel_size, strides, 'same')(x)
    if act is not None:
        x = act(x)
    return x, ch_out


def is_pow_2(n: int) -> bool:
    while n > 1:
        if n % 2: return False
        n //= 2
    return True


def hbit(n: int) -> int:
    return int(np.log2(n))



def build_layers_down(inp_img_size: int = 256, inp_channels: int = 6, batch_size: Optional[int] = None) -> Tuple[Any, Any]:
    assert is_pow_2(inp_img_size), f'input_size = {inp_img_size} is not power of 2'
    assert 32 <= inp_img_size <= 512
    nbits_in = hbit(inp_img_size)
    input_shape = inp_img_size, inp_img_size, inp_channels
    inp = tf.keras.Input(input_shape, batch_size=batch_size, dtype=tf.float32)

# Basic scenario:
#    size  channels
# 0   256         6
# 1   128         8
# 2    64        16
# 3    32        32
# 4    16        64
# 5     8       128
# 6     4       256
# 7     2       512
# 8     1      1024

    ch, ker_sz, stride = 4, 5, 2
    if nbits_in < 8:
        ch *= 2**(8 - nbits_in)
    elif nbits_in > 8:
        ker_sz, stride = 7, 4

    x = inp
    while nbits_in > 1:
        act = None if nbits_in == 1 else tf.nn.relu
        x, ch = block_down(x, stride, ch, ch * 2, ker_sz, act)
        nbits_in -= hbit(stride)
        if nbits_in + hbit(ch) == 10:
            ker_sz, stride = 5, 2
        if nbits_in <= 4:
            ker_sz = 3
    ch *= 2
    x = tf.keras.layers.Conv2D(ch, 2, 1, 'valid')(x)

    out = tf.reshape(x, (-1, ch))

    return inp, out


In [4]:
maps_inp, map_features = build_layers_down()
print(maps_inp, map_features)

2022-11-02 22:39:15.011184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 6), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='tf.reshape/Reshape:0', description="created by layer 'tf.reshape'")


In [ ]:
# `maps_features`: N_components x Feature_dimension. Feature represents embedding extracted by
# backbone CNN from object's maps.
# `graph_pts`: N_points_in_one_level x 3. Contains set of points distributed on sphere which represents
# one level of vertices in the graph.
# `n_verts` = N_points_in_one_level * `n_levels`. Number of vertices in the graph.
# `n_levels` - number of descrete steps of interval: [0, pi), i.e. given h = pi / `n_levels`,
# levels (0, 1, ..., `n_levels` - 1) correspond to values (0, h, ..., (`n_levels` - 1) * h) respectively.
# `edges_dir`: N_components x N_directed_edges. Directed edges of graph with `n_verts` vertices.
# `batch_size` = N_components.
# `inp_pose_size` - Size of pose parameters vector needed to restore 3d position of object' center.
def build_graph_head(maps_features: Any, graph_pts: np.ndarray, n_verts: int, n_levels: int, edges_dir: np.ndarray, batch_size: int, inp_pose_size: int = 6):
    inp_pose = tf.keras.Input((inp_pose_size,), batch_size=batch_size, dtype=tf.float32)

    n_pts, n_edges_dir = len(graph_pts), len(edges_dir)
    pts = tf.convert_to_tensor(graph_pts, dtype=tf.float32)
    edges_1 = tf.convert_to_tensor(edges_dir, dtype=tf.int64)
    edges_2 = tf.reverse(edges_1, [1])
    edges = tf.concat([edges_1, edges_2], axis=0)
    n_edges = n_edges_dir * 2
    edges = tf.tile(edges, (batch_size, 1))
    shift = tf.range(0, batch_size * n_edges, n_edges, dtype=tf.int64)
    shift = tf.repeat(shift, n_edges)[..., tf.newaxis]
    edges += shift

    angle_step = np.pi / n_levels
    angles = tf.range(n_levels, dtype=tf.float32) * angle_step
    angles = tf.reshape(angles, (n_levels, 1))

    # batch_size x Feature_dimension --> batch_size * n_verts x Feature_dimension
    vert_feat_maps = tf.repeat(maps_features, n_verts, axis=0)

    # N_points_in_one_level x 3 --> n_verts x 3
    vert_feat_pts = tf.tile(pts, (n_levels, 1))
    # n_verts x 3 --> batch_size * n_verts x 3
    vert_feat_pts = tf.tile(vert_feat_pts, (batch_size, 1))

    # n_levels x 1 --> n_verts x 1
    vert_feat_angle = tf.repeat(angles, n_pts, axis=0)
    # n_verts x 1 --> batch_size * n_verts x 1
    vert_feat_angle = tf.tile(vert_feat_angle, (batch_size, 1))

    # batch_size x inp_pose_size --> batch_size * n_verts x inp_pose_size
    vert_feat_pose = tf.repeat(inp_pose, n_verts, axis=0)

    # batch_size * n_verts x (Feature_dimension + 3 + 1 + inp_pose_size)
    vert_feat = tf.concat([vert_feat_maps, vert_feat_pts, vert_feat_angle, vert_feat_pose], axis=1)
    vert_feat = tf.convert_to_tensor(vert_feat)
    print(vert_feat.shape)


    graph = tfgnn.GraphTensor.from_pieces(
        context=tfgnn.Context.from_fields(
            features={}
        ),
        node_sets={
            tfgnn.NODES: tfgnn.NodeSet.from_fields(
                sizes=tf.constant([n_verts] * batch_size),
                features={
                    tfgnn.HIDDEN_STATE: vert_feat,
                },
            ),
        },
        edge_sets={
            tfgnn.EDGES: tfgnn.EdgeSet.from_fields(
                sizes=tf.constant([len(edges)] * batch_size),
                adjacency=tfgnn.Adjacency.from_indices(
                    source=(tfgnn.NODES, edges[:, 0]),
                    target=(tfgnn.NODES, edges[:, 1]),
                ),
            ),
        }
    )

    print(graph)

    # graph = tfgnn.keras.layers.NodeSetUpdate(
    #     next_state=tfgnn.keras.layers.NextStateFromConcat(tf.keras.layers.Dense(128)),
    #     context_input_feature=['map_features', 'inp_pose', 'angles', 'pts'],
    # )(graph)

    # graph = GCNHomGraphUpdate(units=128, add_self_loops=True)(graph)
    # graph = GCNHomGraphUpdate(units=128, add_self_loops=True)(graph)
    # graph = GCNHomGraphUpdate(units=64, add_self_loops=True)(graph)
    # graph = GCNHomGraphUpdate(units=64, add_self_loops=True)(graph)

    # return inp_pose, graph
    
    return None, None


In [5]:
n_pts = 1000
n_levels = 100
pts, n_verts, edges = calc_3d_graph_elevated(n_pts, n_levels)

In [37]:
inp_img_size = 256
inp_channels = 6
batch_size = 10
img_inp, features = build_layers_down(inp_img_size, inp_channels, batch_size=batch_size)

In [38]:
inp_pose_size = 6
inp_pose, graph = build_graph_head(features, pts, n_verts, n_levels, edges, batch_size, inp_pose_size)

NameError: name 'build_graph_head' is not defined

In [53]:
class GcnLayer(tf.keras.layers.Layer):
    def __init__(self, A: tf.sparse.SparseTensor, batch_size: int, n_verts: int, units: int, normalize: bool = True, add_self_loops: bool = True):
        super().__init__()
        self.A = A
        self.batch_size = batch_size
        self.n_verts = n_verts
        self.units = units
        self.normalize = normalize
        self.add_self_loops = add_self_loops
    
    def call(self, features: Any):
        normalized_values = features
        normalized_values = tf.reshape(normalized_values, (self.batch_size * self.n_verts, features.shape[-1]))
        invsqrt_deg = tf.constant([])
        if self.normalize:
            in_degree = tf.sparse.reduce_sum(self.A, axis=-1)
            
            if self.add_self_loops:
                in_degree += 1
            
            invsqrt_deg = tf.math.rsqrt(in_degree)
            normalized_values = invsqrt_deg[:, tf.newaxis] * features

        print(tf.shape(self.A), tf.shape(normalized_values))
        pooled = tf.sparse.sparse_dense_matmul(self.A, normalized_values)

        if self.add_self_loops:
            if self.normalize:
                pooled += invsqrt_deg[:, tf.newaxis] * normalized_values
            else:
                pooled += normalized_values

        # pooled = tf.reshape(pooled, (self.batch_size, self.n_verts, -1))
        out = tf.keras.layers.Dense(units=self.units, activation='relu', use_bias=False)(pooled)
        
        return out


In [51]:
# `maps_features`: N_components x Feature_dimension. Feature represents embedding extracted by
# backbone CNN from object's maps.
# `graph_pts`: N_points_in_one_level x 3. Contains set of points distributed on sphere which represents
# one level of vertices in the graph.
# `n_verts` = N_points_in_one_level * `n_levels`. Number of vertices in the graph.
# `n_levels` - number of descrete steps of interval: [0, pi), i.e. given h = pi / `n_levels`,
# levels (0, 1, ..., `n_levels` - 1) correspond to values (0, h, ..., (`n_levels` - 1) * h) respectively.
# `edges_dir`: N_components x N_directed_edges. Directed edges of graph with `n_verts` vertices.
# `batch_size` = N_components.
# `inp_pose_size` - Size of pose parameters vector needed to restore 3d position of object' center.
def build_head_manual(maps_features: Any, graph_pts: np.ndarray, n_verts: int, n_levels: int, edges_dir: np.ndarray, batch_size: int, inp_pose_size: int = 6,
        normalize: bool = True, add_self_loops: bool = True):
    inp_pose = tf.keras.Input((inp_pose_size,), batch_size=batch_size, dtype=tf.float32)
    vert_pts = tf.convert_to_tensor(graph_pts, dtype=tf.float32)
    
    # Build adjacency matrix
    n_edges_dir = len(edges_dir)
    n_edges = n_edges_dir * 2
    edges_1 = tf.convert_to_tensor(edges_dir, dtype=tf.int64)
    edges_2 = tf.reverse(edges_1, axis=[1])
    edges = tf.concat([edges_1, edges_2], axis=0)
    edges = tf.tile(edges, (batch_size, 1))
    shift = tf.range(0, batch_size * n_edges, n_edges, dtype=tf.int64)
    shift = tf.repeat(shift, n_edges)[..., tf.newaxis]
    edges += shift

    A = tf.sparse.SparseTensor(edges, tf.ones(batch_size * n_edges, tf.float32), (batch_size * n_verts, batch_size * n_verts))
    A = tf.sparse.reorder(A)

    angle_step = np.pi / n_levels
    angles = tf.range(n_levels, dtype=tf.float32) * angle_step
    angles = tf.reshape(angles, (n_levels, 1))

    # batch_size x Feature_dimension --> batch_size * n_verts x Feature_dimension
    vert_feat_maps = tf.repeat(maps_features, n_verts, axis=0)

    # n_verts x 3 --> batch_size * n_verts x 3
    vert_feat_pts = tf.tile(vert_pts, (batch_size, 1))

    # n_levels x 1 --> n_verts x 1
    vert_feat_angle = tf.repeat(angles, n_pts, axis=0)
    # n_verts x 1 --> batch_size * n_verts x 1
    vert_feat_angle = tf.tile(vert_feat_angle, (batch_size, 1))

    # batch_size x inp_pose_size --> batch_size * n_verts x inp_pose_size
    vert_feat_pose = tf.repeat(inp_pose, n_verts, axis=0)

    # batch_size * n_verts x (Feature_dimension + 3 + 1 + inp_pose_size)
    vert_feat = tf.concat([vert_feat_maps, vert_feat_pts, vert_feat_angle, vert_feat_pose], axis=1)
    vert_feat = tf.convert_to_tensor(vert_feat)
    print(vert_feat.shape)

    x = vert_feat
    x = GcnLayer(A, batch_size, n_verts, 128, normalize, add_self_loops)(x)
    x = GcnLayer(A, batch_size, n_verts, 128, normalize, add_self_loops)(x)
    x = GcnLayer(A, batch_size, n_verts, 64, normalize, add_self_loops)(x)
    x = GcnLayer(A, batch_size, n_verts, 32, normalize, add_self_loops)(x)
    x = GcnLayer(A, batch_size, n_verts, 16, normalize, add_self_loops)(x)
    x_pos = x

    x = GcnLayer(A, batch_size, n_verts, 8, normalize, add_self_loops)(x)
    x = GcnLayer(A, batch_size, n_verts, 1, normalize, add_self_loops)(x)
    out_rot = x

    x = x_pos
    x = tf.keras.layers.Dense(16, 'relu', use_bias=True)(x)
    x = tf.keras.layers.Dense(16, 'relu', use_bias=True)(x)
    x = tf.reshape(x, (batch_size, n_verts, 16))
    x = tf.reduce_mean(x, axis=1)
    x = tf.keras.layers.Dense(16, 'relu', use_bias=True)(x)
    x = tf.keras.layers.Dense(16, 'relu', use_bias=True)(x)
    x = tf.keras.layers.Dense(3, 'relu', use_bias=True)(x)
    out_pos = x

    return inp_pose, out_pos, out_rot



In [52]:
inp_pose_size = 6
inp_pose, out_pos, out_rot = build_head_manual(features, pts, n_verts, n_levels, edges, batch_size, inp_pose_size)

(1000000, 1034)
reduce_sum Tensor("gcn_layer_9/Shape/Cast:0", shape=(2,), dtype=int32)
!!! reduced !!!
Tensor("gcn_layer_9/Shape_1/Cast:0", shape=(2,), dtype=int32) Tensor("gcn_layer_9/Shape_2:0", shape=(2,), dtype=int32)
!!! done !!!
make_gcn_layer (1000000, 128)
reduce_sum Tensor("gcn_layer_10/Shape/Cast:0", shape=(2,), dtype=int32)
!!! reduced !!!
Tensor("gcn_layer_10/Shape_1/Cast:0", shape=(2,), dtype=int32) Tensor("gcn_layer_10/Shape_2:0", shape=(2,), dtype=int32)
!!! done !!!
make_gcn_layer (1000000, 128)
reduce_sum Tensor("gcn_layer_11/Shape/Cast:0", shape=(2,), dtype=int32)
!!! reduced !!!
Tensor("gcn_layer_11/Shape_1/Cast:0", shape=(2,), dtype=int32) Tensor("gcn_layer_11/Shape_2:0", shape=(2,), dtype=int32)
!!! done !!!
make_gcn_layer (1000000, 64)
reduce_sum Tensor("gcn_layer_12/Shape/Cast:0", shape=(2,), dtype=int32)
!!! reduced !!!
Tensor("gcn_layer_12/Shape_1/Cast:0", shape=(2,), dtype=int32) Tensor("gcn_layer_12/Shape_2:0", shape=(2,), dtype=int32)
!!! done !!!
make_gcn_l

In [ ]:
features

In [ ]:
features.__dict__

In [ ]:
features.numpy()

In [ ]:
tf.keras.backend.get_value(features)

In [ ]:
K = tf.keras.backend
K.eager(K.get_value)(features)

In [ ]:
K.eval(features)

In [ ]:
tf.fill(5, 1)

In [ ]:
tf.ones(5, dtype=tf.int32)

In [14]:
A = tf.sparse.SparseTensor([[0, 0, 2], [0, 0, 3], [0, 2, 3], [0, 4, 3], [0, 4, 2]], [1.0, 1., 1., 1., 1.], (1, 5, 5))
A = tf.sparse.reorder(A)
print(A)

SparseTensor(indices=tf.Tensor(
[[0 0 2]
 [0 0 3]
 [0 2 3]
 [0 4 2]
 [0 4 3]], shape=(5, 3), dtype=int64), values=tf.Tensor([1. 1. 1. 1. 1.], shape=(5,), dtype=float32), dense_shape=tf.Tensor([1 5 5], shape=(3,), dtype=int64))


In [15]:
x: tf.Tensor = tf.sparse.reduce_sum(A, axis=1)
print(x)
x += 1
x

tf.Tensor([[0. 0. 2. 3. 0.]], shape=(1, 5), dtype=float32)


<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[1., 1., 3., 4., 1.]], dtype=float32)>

In [18]:
x = tf.constant([[1.0, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
x = tf.reshape(x, (2, 3, 2))
x

<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[ 1.,  2.],
        [ 3.,  4.],
        [ 5.,  6.]],

       [[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]]], dtype=float32)>

In [19]:
tf.sparse.sparse_dense_matmul(A, x)

InvalidArgumentError: {{function_node __wrapped__SparseTensorDenseMatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Tensor 'b' is not a matrix [Op:SparseTensorDenseMatMul]

In [5]:
x = tf.random.uniform((2, 2, 3), 0, 10, dtype=tf.int32)
x

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[6, 3, 0],
        [8, 6, 7]],

       [[8, 3, 0],
        [9, 7, 1]]], dtype=int32)>

In [12]:
tf.reshape(x, (2, -1))

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[6, 3, 0, 8, 6, 7],
       [8, 3, 0, 9, 7, 1]], dtype=int32)>

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[8, 3, 0],
       [9, 7, 1]], dtype=int32)>

In [13]:
input = tf.random.normal(shape=(3, 12))
k = 2
values, indices  = tf.math.top_k(input, k=k)
print(values.shape)


(3, 2)


In [14]:
input

<tf.Tensor: shape=(3, 12), dtype=float32, numpy=
array([[ 0.3277132 , -0.08097047, -1.4443549 , -0.37999842, -0.51421654,
        -1.3463523 ,  1.169286  ,  0.40608847, -1.3081495 , -0.17982765,
         2.1462622 , -1.832191  ],
       [-0.5616635 , -0.8308778 ,  0.07133581,  0.22717148, -0.49391243,
        -0.83485305, -0.14846714,  1.1375853 , -1.4950925 ,  0.72516304,
         0.3868215 , -1.0906208 ],
       [-1.3250701 , -1.0344608 ,  1.29745   , -1.3784496 , -0.6767564 ,
        -0.16289645,  0.24838132,  0.23087345,  0.79717726, -1.4853148 ,
         0.42025313, -0.39925814]], dtype=float32)>

In [15]:
values

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[2.1462622 , 1.169286  ],
       [1.1375853 , 0.72516304],
       [1.29745   , 0.79717726]], dtype=float32)>

In [16]:
indices

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  6],
       [ 7,  9],
       [ 2,  8]], dtype=int32)>

In [19]:
x

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[6, 3, 0],
        [8, 6, 7]],

       [[8, 3, 0],
        [9, 7, 1]]], dtype=int32)>

In [20]:
x[..., :2]

<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[6, 3],
        [8, 6]],

       [[8, 3],
        [9, 7]]], dtype=int32)>